In [6]:
import pandas as pd



In [27]:
path = "/Users/liangtelkamp/Documents/GitHub/un-sdd/processing/tests/test2.xlsx"
df = pd.read_excel(path)
df


,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,Table 3: Number of acreage under irrigation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,OVERALL,NaN,Sub county,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,Chepalungu,NaN,NaN,NaN,Bomet Central,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,Male,NaN,Female,NaN,Male,NaN,Female,NaN
4,NaN,NaN,N,%,N,%,N,%,N,%,N,%
5,What is the average size of land you own that ...,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
6,NaN,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
7,NaN,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
8,NaN,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
9,NaN,NaN,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%


In [31]:
import pandas as pd

def concatenate_multirow_headers_auto(df, sep=' | ', max_header_rows=10, threshold=0.5, special_symbols=None):
    if special_symbols is None:
        special_symbols = ['N', '%']  # add more symbols if needed
    
    n_rows = df.shape[0]
    
    # Step 1: Find the first row with >= threshold non-NaN values
    for i in range(min(max_header_rows, n_rows)):
        non_nan_fraction = df.iloc[i].notna().mean()
        if non_nan_fraction >= threshold:
            first_header_row = i
            break
    else:
        first_header_row = 0  # fallback if none meet threshold

    # Step 2: Include subsequent rows that are mostly NaN or contain special symbols
    last_header_row = first_header_row
    for i in range(first_header_row + 1, n_rows):
        row = df.iloc[i].astype(str)
        contains_symbol = any(symbol in row.values for symbol in special_symbols)
        mostly_empty = row.notna().mean() < threshold
        if mostly_empty or contains_symbol:
            last_header_row = i
        else:
            break

    # Step 3: Concatenate header rows
    headers = df.iloc[:last_header_row + 1].fillna(method='ffill', axis=1)
    concatenated_headers = headers.apply(lambda col: sep.join([str(i) for i in col if str(i) != 'nan']), axis=0)
    
    # Step 4: Set headers and drop header rows from data
    df.columns = concatenated_headers
    df = df.iloc[last_header_row + 1:].reset_index(drop=True)
    
    return df

# Usage example
df = pd.read_excel(path, header=None)
df_clean = concatenate_multirow_headers_auto(df)
print(df_clean)


         Table 3: Number of acreage under irrigation  \
0  What is the average size of land you own that ...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   
5                                                NaN   

  Table 3: Number of acreage under irrigation  \
0                                 0 - 2 acres   
1                                 2 - 5 acres   
2                                5 - 10 acres   
3                          More than 10 acres   
4                                         NaN   
5                                       Total   

  Table 3: Number of acreage under irrigation | OVERALL | N  \
0                                                 22          
1                                                  6          
2                                                  1          
3          

/var/folders/lm/8lf06d816nv8ym2rv97d1v580000gn/T/ipykernel_80823/3456906327.py:30: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  headers = df.iloc[:last_header_row + 1].fillna(method='ffill', axis=1)


In [ ]:
import pandas as pd

def auto_merge_empty_columns(df, sep=' | '):
    """
    Automatically merge columns that are all NaN into the previous non-empty column.
    """
    df = df.copy()
    # Print first column values
    print(df.iloc[:, 0])
    
    # If those are all NaN, merge them into the previous column
    if df.iloc[:, 0].isna().all():
        print("First column is all NaN")
        # Get name of first column
        first_name = (df.columns[0])
        # Concatenate the text of the first column with the text of the second column
        # Drop the second column
    return df

# Suppose df_clean is your DataFrame after cleaning headers
df_merged = auto_merge_empty_columns(df_clean)
df_merged


0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
Name: (What is the average size of land you own that is currently under irrigation?,), dtype: object
First column is all NaN
('What is the average size of land you own that is currently under irrigation?',)


,What is the average size of land you own that is currently under irrigation?,0 - 2 acres,22,2.8%,4,2.2%,10,3.8%,3,1.7%,5,2.9%
0,NaN,2 - 5 acres,6,.8%,2,1.1%,2,.8%,0,0.0%,2,1.2%
1,NaN,5 - 10 acres,1,.1%,0,0.0%,0,0.0%,0,0.0%,1,.6%
2,NaN,More than 10 acres,0,0.0%,0,0.0%,0,0.0%,0,0.0%,0,0.0%
3,NaN,NaN,760,96.3%,176,96.7%,251,95.4%,170,98.3%,163,95.3%
4,NaN,Total,789,100.0%,182,100.0%,263,100.0%,173,100.0%,171,100.0%
